In [1]:
!ls

activate_environment.sh  models		Optimizers	README.md	 tests
conda_env.yml		 nnfl		poetry.lock	README.rst	 wandb
First_notebook.ipynb	 nnfl.egg-info	pyproject.toml	start_off.ipynb


In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
from fastai.vision import *
from torch.optim import Optimizer
from Optimizers.lookahead import *
from torchcontrib.optim import SWA
import wandb
from wandb.fastai import WandbCallback

In [4]:
PROJECT_PATH = Path.cwd()
MODELS = PROJECT_PATH/'models'

MODELS.mkdir(exist_ok=True)

In [5]:
def print_all(*args):
    for i in args:
        print(i)

In [6]:
def get_mnist_data(bs=16):
    PATH = Path("/home/ubuntu/.fastai/data/mnist_png")
    data = (ImageList.from_folder(path=PATH)
            .split_by_folder(train='training', valid='testing')
            .label_from_folder()
            .databunch(bs=bs))
    return data

In [7]:
def get_cifar10_data(bs=16):
    pad4 = RandTransform(tfm=pad, kwargs={'padding':4, 'mode':'zeros'}, is_random=False, p=1.0, use_on_y=False)
    crop32 = RandTransform(tfm=crop, kwargs={'row_pct':(0.4,0.6), 'col_pct':(0.4,0.6), 'size':32}, p=1.0, use_on_y=False)
    tfms = [pad4, crop32]
    PATH = Path("/home/ubuntu/.fastai/data/cifar10")
    data = (ImageList.from_folder(path=PATH)
            .split_by_folder(train='train', valid='test')
            .label_from_folder()
            .transform((tfms, []))
            .databunch(bs=bs))
    return data

### Check if everything works

In [8]:
data = get_mnist_data()
# data.show_batch()

def get_simple_cnn(pretrained=False):
    return simple_cnn([3, 4, 2])

learn = cnn_learner(data=data, base_arch=get_simple_cnn, opt_func=LookaheadSGD)
learn.metrics.append(accuracy)
learn.fit_one_cycle(1, 0.003)
learn.save(MODELS/'mnist_sample.pkl')
learn.load(MODELS/'mnist_sample.pkl')

KeyboardInterrupt: 

### Running the experiment

In [8]:
wandbRecorder = partial(WandbCallback, input_type='images')

In [9]:
optimizers = [LookaheadSGD, optim.SGD, optim.AdamW, optim.RMSprop, LookaheadAdamW] # Along with SWA which needs to be separately dealt with
experiment_names = {LookaheadSGD:'Lookahead_SGD', optim.SGD:'SGD', optim.AdamW:'AdamW', 
                        optim.RMSprop:'RMSProp', LookaheadAdamW:'LookaheadAdamW'}

lookahead_sgd_params = dict(alpha=0.5, k=5) # lr = 0.1
sgd_params = dict(momentum=0.9, weight_decay=0.001) # lr = 0.05
adamW_params = dict(weight_decay=0.3) # lr = 0.001
rmsprop_params = dict(weight_decay=0.001) # lr = 0.01
lookahead_adamw_params = dict(weight_decay=0.3, k=5, alpha=0.5) # lr = 0.1

params_list = [lookahead_sgd_params, sgd_params, adamW_params, rmsprop_params, lookahead_adamw_params]

params_dict = {LookaheadSGD:lookahead_sgd_params, optim.SGD:sgd_params, optim.AdamW:adamW_params,
              optim.RMSprop:rmsprop_params, LookaheadAdamW:lookahead_adamw_params}
lr_dict = {LookaheadSGD:0.1, optim.SGD:0.05, optim.AdamW:0.001,
              optim.RMSprop:0.01, LookaheadAdamW:0.1}
epochs_dict = {LookaheadSGD:20, optim.SGD:20, optim.AdamW:20,
              optim.RMSprop:20, LookaheadAdamW:20}

In [10]:
wandb.init()

W&B Run: https://app.wandb.ai/akashpalrecha/lookahead/runs/akfzk743

In [11]:
def fit_and_record(epochs=None, lr=None, opt_func=LookaheadSGD, experiment_name=None, 
                   data=None, base_arch=models.resnet18, pretrained=False):
    np.random.seed(42)
    # Initialize Training
    if experiment_name is None:
        experiment_name = experiment_names[opt_func]
    if data is None:
        data = get_cifar10_data(128)
    if epochs is None:
        epochs = epochs_dict[opt_func]
    if lr is None:
        lr = lr_dict[opt_func]
    # Setting optimizer parameters correctly
    opt_func = partial(opt_func, **params_dict[opt_func])
    
    print_all(data, opt_func, epochs, lr, experiment_name, base_arch)
    
    wandb.init(project='lookahead', name=experiment_name)
    learn = cnn_learner(data=data, base_arch=base_arch, opt_func=opt_func, pretrained=pretrained, callback_fns=WandbCallback)
    learn.metrics.append(accuracy)
    learn.fit(epochs, lr)
    learn.save(MODELS/experiment_name)

In [12]:
for optimizer in optimizers:
    fit_and_record(epochs=1, opt_func=optimizer, pretrained=True)

ImageDataBunch;

Train: LabelList (50000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Valid: LabelList (10000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Test: None
functools.partial(<class 'Optimizers.lookahead.LookaheadSGD'>, alpha=0.5, k=5)
1
0.1
Lookahead_SGD
<function resnet18 at 0x7fbf4100ce60>


epoch,train_loss,valid_loss,accuracy,time
0,1.269825,1.105289,0.609100,00:10


Better model found at epoch 0 with valid_loss value: 1.105289101600647.
Loaded best saved model from /home/ubuntu/personal/Lookahead/wandb/run-20191113_170026-6le2fxar/bestmodel.pth
ImageDataBunch;

Train: LabelList (50000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Valid: LabelList (10000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Test: None
functools.partial(<class 'torch.optim.sgd.SGD'>, momentum=0.9, weight_decay=0.001)
1
0.05
SGD
<function resnet18 at 0x7fbf4100ce60>


epoch,train_loss,valid_loss,accuracy,time
0,1.747736,1.442691,0.481700,00:09


Better model found at epoch 0 with valid_loss value: 1.4426907300949097.
Loaded best saved model from /home/ubuntu/personal/Lookahead/wandb/run-20191113_170042-bmpi9iu9/bestmodel.pth
ImageDataBunch;

Train: LabelList (50000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Valid: LabelList (10000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Test: None
functools.partial(<class 'torch.optim.adamw.AdamW'>, weight_decay=0.3)
1
0.001
AdamW
<function resnet18 at 0x7fbf4100ce60>


...

Better model found at epoch 0 with valid_loss value: 1.2313683032989502.
Loaded best saved model from /home/ubuntu/personal/Lookahead/wandb/run-20191113_170055-0zkoo7di/bestmodel.pth
ImageDataBunch;

Train: LabelList (50000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Valid: LabelList (10000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Test: None
functools.partial(<class 'torch.optim.rmsprop.RMSprop'>, weight_decay=0.001)
1
0.01
RMSProp
<function resnet18 at 0x7fbf4100ce60>


epoch,train_loss,valid_loss,accuracy,time
0,1.257879,1.231961,0.584000,00:10


Better model found at epoch 0 with valid_loss value: 1.2319611310958862.
Loaded best saved model from /home/ubuntu/personal/Lookahead/wandb/run-20191113_170110-vn6wm1tc/bestmodel.pth
ImageDataBunch;

Train: LabelList (50000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Valid: LabelList (10000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
horse,horse,horse,horse,horse
Path: /home/ubuntu/.fastai/data/cifar10;

Test: None
functools.partial(<class 'Optimizers.lookahead.LookaheadAdamW'>, weight_decay=0.3, k=5, alpha=0.5)
1
0.1
LookaheadAdamW
<function resnet18 at 0x7fbf4100ce60>


epoch,train_loss,valid_loss,accuracy,time
0,1.459955,1.263729,0.545800,00:12


Better model found at epoch 0 with valid_loss value: 1.2637289762496948.
Loaded best saved model from /home/ubuntu/personal/Lookahead/wandb/run-20191113_170124-12mv9hmm/bestmodel.pth


### Making SWA Work

In [13]:
class SWAStep(LearnerCallback):
    def on_train_end(self, **kwargs):
        self.learn.opt.opt.swap_swa_sgd()

In [ ]:
learn = cnn_learner(data=data, base_arch=base_arch, opt_func=opt_func, pretrained=pretrained, callback_fns=WandbCallback)
learn.metrics.append(accuracy)
learn.fit(epochs, lr)